In [15]:
## if needed, install the libraries using pip install
## pip install pandas

In [16]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import numpy as np
from sklearn.metrics import mean_squared_error
#import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
#from bokeh.io import output_notebook
#from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Legend, ColumnDataSource
import unittest
import os
from git import Repo


In [17]:
#create data exceptions, here the most common
class DataError(Exception): #dataerror is a subclass of Exception class
    pass
#we are now defining subclass of dataerror which will represent a specific error
class FileLoadingError(DataError):
    pass

class DataProcessingError(DataError):
    pass

## HANDLING DATA

In [18]:
# create a class for handling data: manage, manipulate, and organize data
class DataHandler:
    def __init__(self, db_name="main.db"):
        self.db_name = db_name 
        """
        if nothing is specified, datahandler uses "main" as db.
        without this part of code we should set the db name as parameter of each method
        
        """
    
    
    def load_data(self, csv_file, table_name):
        """
        Load a csv file into a sqlite db table:
        - csv_file: path of the csv to be loaded.
        - table_name: name of the table in the db
        """
        try:
            # connect to the sqlite db
            conn = sqlite3.connect(self.db_name)
            
            # load the csv file into a df using panda 
            df = pd.read_csv(csv_file)

            # write the df in the sqlite table, replacing any table whit same name
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"Wow you have loaded a new table '{table_name}' to your MAIN database, great job!")
            
            conn.close() #connection closed
            
        #insert most common exception handling, provides clearer error messages
        except FileNotFoundError as e:
            raise FileLoadingError(f"File {csv_file} not found, check your path or filename") from e
        except Exception as e:
            raise DataProcessingError("Error during data loading") from e
        except pd.errors.EmptyDataError as e:
            raise DataProcessingError(f"No data found in {csv_file}") from e

    
    def get_data(self, table_name):
        """
        get data from a sqlite database table.
        """
        try:
            engine = create_engine(f'sqlite:///{self.db_name}')
            conn = engine.connect()
            data = pd.read_sql(f"SELECT * FROM {table_name}", conn)
            #print(f"Congrats! Your data object is now available")
            #conn.close()
            return data
        except Exception as e:
            raise DataProcessingError("Error retrieving data from the database!") from e
        finally:
            if conn:  #"if conn:" helps to avoid errors when conn is not loaded properly
                conn.close()



## LEAST SQUARES METHOD
### Find the 4 ideal functions that are closest to our train data

In [19]:


def lsm_ideal_function(training_data, ideal_f):
    """
    find the 4 ideal functions that fit the most using the least squares method. f= function
    iterate over the 4 training columns finding for each of them the ideal function that fits the most.
    
    """
    selected_f = [] #initialize a list to store the ideal functions
    
    for i in range(1, 5):  #the first column is not on scope, run from 1 to 4 skipping 0
        min_error = None
        best_f = None
        for f in ideal_f.columns[1:]: #get the name of the columns skipping the first one on position 0, "x"
            error = mean_squared_error(training_data.iloc[:, i], ideal_f[f])
            if min_error is None or error < min_error:
                min_error = error
                best_f = f
        selected_f.append(best_f)
    return selected_f



## MAPPING THE INDIVIDUAL TEST DATA TO THE SELECTED IDEAL FUNCTIONS
### The criteria is that the existing maximum deviation of the calculated regression does not exceed the largest deviation between training dataset (A) and the ideal function (C) chosen for it by more than factor sqrt(2)

In [20]:
def map_test_data(test_data, selected_f, ideal_f, training_data, max_dev_factor=np.sqrt(2)):
    """
    Considering the maximum deviation between training data and the 4 ideal function chosen for them
    a sqrt(2) tollerance factor must be applied
    We then will map the test data with the ideal function with deviation under the theshold
    
        test_data: dataframe with test data, only two columns x and y.
        selected_f: list of selected ideal functions
        ideal_f: dataframe of fifty ideal functions
        training_data: training dataset
        max_dev_factor: tolerance factor

    Returns a dataframe with test data, y_values of ideal function, deviation parameter and selected ideal function label
    """
    best_f = []  # initialize the list to collect the ideal functions
    max_dev = {}  # initialize a dictonary in order to contain the deviation for ideal function
    
    #Calculate the maximum deviation between training data and ideal function
    max_allowed_dev = {}
    for i, func in enumerate(selected_f, start=1):
        y_train = training_data.iloc[:, i]
        y_ideal = ideal_f[func]
        max_dev = np.max(np.abs(y_train - y_ideal))  # deviations 
        max_allowed_dev[func] = max_dev * max_dev_factor  # theshold
    print (max_allowed_dev)

    #we are now searching for the ideal function for the test data from the 4 selected
    results = []
    for _, row in test_data.iterrows():
        x_test = row['x']
        y_test = row['y']
        for func in selected_f:
            y_ideal = ideal_f[ideal_f['x'] == x_test][func].values[0] #ensure that we are considering same x values 
            dev = abs(y_test - y_ideal)
            if dev <= max_allowed_dev[func]:
                results.append([x_test, y_test, y_ideal, dev, max_allowed_dev[func],func])
        best_f= pd.DataFrame(results, columns=['x', 'y_test', 'y_ideal', 'deviation','max_allowed_dev','ideal_func'])

    # excel export
    best_f.to_excel("best_function.xlsx", index=False)
        
    return best_f
    



## VISUALIZE THE DATA

## Train Data vs Selected Ideal Functions

In [27]:


def visualize_selected_functions(training_data, selected_f, ideal_f):
    """
    Visualize training data overlapped to selected functions
    """
    output_notebook()  #use this line of code if using Jupiter
    colors = ['blue', 'darkorange', 'green', 'lightseagreen' ] #set some colors
    colors2 = ['lightblue', 'gold', 'limegreen', 'skyblue' ]

    p = figure(width=700, height=600, title="Training Data and Selected Ideal Functions", 
               x_axis_label='x_values', y_axis_label='y_values')


    # training data
    for i in range(4):
        source = ColumnDataSource(data=dict(x=training_data.iloc[:, 0], y=training_data.iloc[:, i+1]))
        p.scatter('x', 'y', source=source, size=2,color=colors[i], legend_label=f'Training Y{i+1}')
    

    # selected ideal functions
    for i, func in enumerate(selected_f):
        source = ColumnDataSource(data=dict(x=ideal_f.iloc[:, 0], y=ideal_f[func]))
        p.line('x', 'y', source=source, line_width=2, color=colors2[i], legend_label=f'Selected {func}')

    #add a custom legend
    p.legend.title = "Legend"
    p.legend.label_text_font_size = "7pt"
    p.legend.location = "top"
    p.add_layout(p.legend[0], 'right')
    p.legend.orientation = 'vertical'
    p.legend.ncols = 2

    show(p)



## Test deta VS Best Selected Functions

In [28]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import output_notebook

def visualize_best_functions( selected_f, ideal_f, best_f):
    """
    Visualize test data overlapped to the best selected functions
    """
    output_notebook()  #use this line of code if using Jupiter
    colors = ['blue', 'darkorange', 'green', 'lightseagreen' ] #set some colors
    colors2 = ['lightblue', 'gold', 'limegreen', 'skyblue' ]
    
    p = figure(width=600, height=600, title="Test Data and Best Ideal Functions", 
               x_axis_label='x_values', y_axis_label='y_values')
    
    # test data
    # Map the color for unique values of ideal_func
    unique_funcs = best_f['ideal_func'].unique()
    color_map = {'y42': 'blue', 'y41': 'darkorange', 'y11': 'green', 'y48': 'lightseagreen'}

    # add color column to df
    best_f['color'] = best_f['ideal_func'].map(color_map)
    
    source = ColumnDataSource(best_f)
    p.scatter('x', 'y_test', source=source, size=2, color= 'color', legend_label=f'Test Y{1}')
    

    # plot the selected ideal function
    for i, func in enumerate(selected_f):
        source = ColumnDataSource(data=dict(x=ideal_f.iloc[:, 0], y=ideal_f[func]))
        p.line('x', 'y', source=source, line_width=2, color=colors2[i], legend_label=f'Selected {func}')

    #add a custom legend
    p.legend.title = "Legend"
    p.legend.label_text_font_size = "7pt"
    p.legend.location = "top"
    p.add_layout(p.legend[0], 'right')
    p.legend.orientation = 'vertical'
    p.legend.ncols = 1

    show(p)



## EXECUTE THE PROGRAM

In [29]:
# Main function to execute the program
def main():
    handler = DataHandler()     #initialize data handlers
    training_handler = handler.load_data (r'C:\Users\shero\Documents\UNIVERSITA\CORSI\3. PYTHON\SCRIPT\train.csv', table_name='train_tab')
    test_handler = handler.load_data (r'C:\Users\shero\Documents\UNIVERSITA\CORSI\3. PYTHON\SCRIPT\test.csv', table_name='test_tab')
    ideal_handler = handler.load_data (r'C:\Users\shero\Documents\UNIVERSITA\CORSI\3. PYTHON\SCRIPT\train.csv', table_name='train_tab')

    #get the data from the database
    training_data = handler.get_data("train_tab")
    test_data = handler.get_data("test_tab")
    ideal_functions = handler.get_data("ideal_tab")
    
    #Find the ideal functions using least squares method
    selected_functions = lsm_ideal_function(training_data, ideal_functions)
    print("The selected ideal functions are:", selected_functions)
    #Visualize the training data with the ideal functions
    visualize_selected_functions(training_data, selected_functions, ideal_functions)

    #Map train data, if not mentioned, factor = np.sqrt(2): (test_data, selected_f, ideal_f, training_data, max_dev_factor=np.sqrt(2))
    best_functions = map_test_data(test_data, selected_functions, ideal_functions, training_data)
    print(best_functions)

    #Visualize best selected functions
    visualize_best_functions(selected_functions, ideal_functions, best_functions)
    
    # to find the test observation not in best function dataframe, we can use merge
    merged = test_data.merge(best_functions[['x']], on='x', how='left', indicator=True)

    #and select the row that are only in merged data
    not_present = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    #save these obs in another excel file
    not_present.to_excel("unmapped_test_data.xlsx", index=False)
    print("Two excel output created: 1. best_function.xlsx contains the mapped test data; 2. unmapped_test_data.xlsx containing the y_test for which the deviation criteria was not met. ")
if __name__ == "__main__":
    main()

Wow you have loaded a new table 'train_tab' to your MAIN database, great job!
Wow you have loaded a new table 'test_tab' to your MAIN database, great job!
Wow you have loaded a new table 'train_tab' to your MAIN database, great job!
The selected ideal functions are: ['y42', 'y41', 'y11', 'y48']


Loading BokehJS ...

{'y42': 0.7014046721030611, 'y41': 0.7038583326337785, 'y11': 0.7056020579561832, 'y48': 0.7067413342598947}
               x     y_test    y_ideal  deviation  max_allowed_dev ideal_func
0   1.750000e+01  34.161040  34.512188   0.351148         0.703858        y41
1   3.000000e-01   1.215102   0.747760   0.467342         0.703858        y41
2   8.000000e-01   1.426456   1.958678   0.532222         0.703858        y41
3   8.000000e-01   1.426456   0.800000   0.626456         0.705602        y11
4   1.400000e+01  -0.066506   0.067726   0.134233         0.706741        y48
5  -1.500000e+01  -0.205363   0.247008   0.452371         0.706741        y48
6   5.800000e+00  10.711373  11.367699   0.656326         0.703858        y41
7  -1.980000e+01 -19.915014 -19.800000   0.115014         0.705602        y11
8   1.890000e+01  19.193245  18.900000   0.293245         0.705602        y11
9   8.800000e+00  -0.726051  -0.237211   0.488840         0.706741        y48
10 -9.500000e+00  -9.652251  -9.5

Loading BokehJS ...

Two excel output created: 1. best_function.xlsx contains the mapped test data; 2. unmapped_test_data.xlsx containing the y_test for which the deviation criteria was not met. 


## UNIT TEST

In [30]:
# Unit test  for LSM 
class TestLSMIdealFunction(unittest.TestCase):
    
    def test_lsm_ideal_function(self):
        # creation simple data for testing
        training_data = pd.DataFrame({
            'x': np.linspace(0, 10, 5),
            'Y1': np.array([1, 2, 3, 4, 5]),
            'Y2': np.array([2, 3, 4, 5, 6]),
            'Y3': np.array([3, 4, 5, 6, 7]),
            'Y4': np.array([4, 5, 6, 7, 8])
        })
        # creation of ideal function same as the testing data so its easier to control the result
        ideal_f = pd.DataFrame({
            'x': np.linspace(0, 10, 5),
            'F1': np.array([1, 2, 3, 4, 5]),
            'F2': np.array([2, 3, 4, 5, 6]),
            'F3': np.array([3, 4, 5, 6, 7]),
            'F4': np.array([4, 5, 6, 7, 8])
        })
        
        # expected values
        expected = ['F1', 'F2', 'F3', 'F4']
        result = lsm_ideal_function(training_data, ideal_f)
        self.assertEqual(result, expected)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.066s

OK


## VERSION CONTROL

In [6]:
#pip install GitPython
import os
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = r"C:\Program Files\Git\bin\git.exe"

from git import Repo

In [14]:
import os
from git import Repo


local_path = "Programming-with-Python.git"
repo_url = "https://github.com/sherony/Programming-with-Python.git" # set the local path and repository


repo = Repo.clone_from(repo_url, local_path, branch="develop") #clone repository
repo.git.checkout("develop") #checkout the develop branch
repo.git.add(".") # Put the changes in staging
repo.index.commit("added new function to improve automation") # create a commit for team collaborators

origin = repo.remote(name="origin") # push the changes on develop
origin.push("develop")



GitCommandError: Cmd('C:\Program Files\Git\bin\git.exe') failed due to: exit code(128)
  cmdline: C:\Program Files\Git\bin\git.exe push --porcelain -- origin develop
  stderr: 'error: failed to execute prompt script (exit code 1)
fatal: could not read Username for 'https://github.com': No such file or directory'

## ANCILLARY CODE

#### HANDLING DATA

In [ ]:
import os
#dir = r'C:\Users\shero\Documents\UNIVERSITA\CORSI\3. PYTHON\SCRIPT'
dir = os.getcwd()
print (dir)

In [ ]:
#load the csv to sql table
if __name__ == "__main__":
    handler = DataHandler()
    handler.load_data ( os.path.join(dir, "ideal.csv") , table_name='ideal_tab')
    handler.load_data ( os.path.join(dir, "test.csv") , table_name='test_tab')
    handler.load_data ( os.path.join(dir, "train.csv") , table_name='train_tab')

In [ ]:
#save the data in 3 different dataset
if __name__ == "__main__":
    handler = DataHandler()
    df_train = handler.get_data('train_tab')
    df_ideal = handler.get_data('ideal_tab')
    df_test = handler.get_data('test_tab')


In [ ]:
print(df_train.head())  # First rows only

#### LEAST SQUARES METHOD
##### Find the 4 ideal functions that are closest to our train data

In [ ]:
selected = lsm_ideal_function(df_train, df_ideal)
print("The selected ideal functions are:", selected)

#### GRAPH

##### Plotting the training data

In [ ]:
# Creating the graph using bokeh
output_notebook()  #use this line of code if using Jupiter

p = figure(width=600, height=600,title="Training Data", 
           x_axis_label='x_values', y_axis_label='y_values')


colors = ['blue', 'darkorange', 'green', 'lightseagreen' ]
colors2 = ['lightblue', 'gold', 'limegreen', 'skyblue' ]

# prot training data
for i in range(4):
    source = ColumnDataSource(data=dict(x=df_train.iloc[:, 0], y=df_train.iloc[:, i+1]))
    p.scatter('x', 'y', source=source, size=1, color=colors[i], legend_label=f'Training Y{i+1}')
    


# add a custom legend
# p.legend.location = "bottom_right"
p.legend.title = "Legend"
p.legend.label_text_font_size = "7pt"
p.legend.location = "top"
p.add_layout(p.legend[0], 'right')
p.legend.orientation = 'vertical'
p.legend.ncols = 1

show(p)

##### Plotting the ideal functions

In [ ]:
# Creating the graph using bokeh
output_notebook()  #use this line of code if using Jupiter

p = figure(width=600, height=600,title="Selected Ideal Functions", 
           x_axis_label='x_values', y_axis_label='y_values')
    

# selected ideal functions
for i, func in enumerate(selected):
    source = ColumnDataSource(data=dict(x=df_ideal.iloc[:, 0], y=df_ideal[func]))
    p.line('x', 'y', source=source, line_width=2, color=colors2[i], legend_label=f'Selected {func}')

# add a custom legend
#p.legend.location = "bottom_right"
p.legend.title = "Legend"
p.legend.label_text_font_size = "7pt"
p.legend.location = "top"
p.add_layout(p.legend[0], 'right')
p.legend.orientation = 'vertical'
p.legend.ncols = 1

show(p)

##### Plotting the train data and the ideal functions

In [ ]:
# Creating the graph using bokeh
output_notebook()  #use this line of code if using Jupiter

p = figure(width=700, height=600,title="Training Data and Selected Ideal Functions", 
           x_axis_label='x_values', y_axis_label='y_values')


# training data
for i in range(4):
    source = ColumnDataSource(data=dict(x=df_train.iloc[:, 0], y=df_train.iloc[:, i+1]))
    p.scatter('x', 'y', source=source, size=2, color=colors[i], legend_label=f'Training Y{i+1}')
    

# selected ideal functions
for i, func in enumerate(selected):
    source = ColumnDataSource(data=dict(x=df_ideal.iloc[:, 0], y=df_ideal[func]))
    p.line('x', 'y', source=source, line_width=2, color=colors2[i], legend_label=f'Selected {func}')

#add a custom legend
#p.legend.location = "bottom_right"
p.legend.title = "Legend"
p.legend.label_text_font_size = "7pt"
p.legend.location = "top"
p.add_layout(p.legend[0], 'right')
p.legend.orientation = 'vertical'
p.legend.ncols = 2

show(p)

##### Plotting test data

In [ ]:
# Creating the graph using bokeh
output_notebook()  #use this line of code if using Jupiter

p = figure(width=600, height=600,title="Pilot Data", 
           x_axis_label='x_values', y_axis_label='y_values')


colors = ['blue', 'darkorange', 'green', 'lightseagreen' ]
colors2 = ['lightblue', 'gold', 'limegreen', 'skyblue' ]

# prot test data
source = ColumnDataSource(data=dict(x=df_test.iloc[:, 0], y=df_test.iloc[:, 1]))
p.scatter('x', 'y', source=source, size=5, color=colors[1], legend_label=f'Test observations')
    


# add a custom legend
p.legend.click_policy = "hide" #interactive graph
p.legend.title = "Legend"
p.legend.label_text_font_size = "7pt"
p.legend.location = "top"
p.add_layout(p.legend[0], 'right')
p.legend.orientation = 'vertical'
p.legend.ncols = 1

show(p)